In [ ]:
import sys, subprocess, pkg_resources
required = {'numpy','lvlspy','matplotlib'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed
if missing:
    subprocess.check_call([sys.executable, '-m','pip','install','--quiet',*missing])


import numpy as np
import lvlspy.level as lv
import lvlspy.spcoll as lc
import lvlspy.species as ls
import lvlspy.transition as lt
import matplotlib.pyplot as plt

The API can be used for equilibrium calculations. Let's create a non-real example and evolve it in time at constant temperature

In [ ]:
levs = [lv.Level(0,1),lv.Level(100,1),lv.Level(500,3),lv.Level(200,4),lv.Level(300,3)]

my_species = ls.Species('my_species',levels = levs)

my_levs = my_species.get_levels()

for i in range(len(my_levs)-1,0,-1):
    for j in range(i-1,-1,-1):
        t = lt.Transition(my_levs[i],my_levs[j],100.)
        my_species.add_transition(t)

T = 2e+9 #temperature in Kelvin

rate_matrix = my_species.compute_rate_matrix(T)

def f_vector(y_dt,y_i,A):
    return np.matmul(A,y_dt) - y_i

tol = 1.e-6 #convergence tolerance on newton raphson solver 
t = np.linspace(0,1,1000)
dt = t[1] - t[0]
y = np.array([1.,0,0,0,0])
A = np.identity(5) - dt*rate_matrix
y_dt = y
y = y_dt
Y = np.empty((5,len(t)))
Y[:,0] = y

for i in range(1,len(t)): 
    delta = np.ones(5)
    while max(delta) > tol:
        delta = np.linalg.solve(A,-f_vector(y_dt,y,A))
        y_dt = y_dt + delta
    Y[:,i] = y_dt
    y = y_dt

eq_probs = my_species.compute_equilibrium_probabilities(T)

plt.figure(figsize=(12,12))
for i in range(len(my_levs)):
    plt.plot(t,Y[i,:],label = str(i) + ' level')
        
for i in range(len(eq_probs)):
    plt.plot(1, eq_probs[i], 'x')

plt.legend()
plt.show()

In [ ]:
fugacities = np.empty((Y.shape[0], Y.shape[1]))

for i in range(len(my_levs)):
    fugacities[i, :] = Y[i, :] / eq_probs[i]
    
plt.figure(figsize=(12,12))
for i in range(len(my_levs)):
    plt.plot(t,fugacities[i,:],label = str(i) + ' level')
    
plt.xlim([0,0.1])